# 4. Data Transformation - 연봉 데이터 다루기
## 4-1. 학습목표: 데이터 변환의 고급 기술을 습득합니다.
1. 데이터를 다양한 방법으로 합치고 변환할 수 있다.
2. 데이터의 스케일을 변환할 수 있다.
3. 카테고리형 데이터를 숫자형태로 변환할 수 있다.
4. 데이터의 차원축소를 할 수 있다.

## 4-2. Data merge(데이터 병합)
1. merge와 join의 특징에 대해 더 알아 봅시다.
2. pandas에서는 merge 함수를 사용하여 SQL 스타일의 조인을 수행할 수 있습니다.

In [1]:
import pandas as pd
import numpy as np

In [2]:
# 병합 실습용 데이터프레임 생성

df_a = pd.DataFrame({'key': ['a', 'b', 'c', 'd', 'e'], 'num_a': [1, 2, 3, 4, 5]})
df_b = pd.DataFrame({'key': ['a', 'b', 'd', 'f', 'g'], 'num_b': [11, 15, 35, 45, 55]})
df_c = pd.DataFrame({'key': ['f', 'g', 'h', 'i', 'j'], 'num_a': [6, 7, 8, 9, 0]})

### 4-2-1.concat() 함수
- pd.concat(axis = int) = 데이터를 int 방향으로 결합; axis 생략시 axis = 0
- 병합할 데이터프레임에 서로 다른 컬럼명이 있을 경우, 컬럼 추가
- 결합방향의 인덱스나 컬럼은 재정렬되지 않고 덧붙여짐에 주목

In [9]:
# axis = 0 혹은 생략의 경우 인덱스는 재정렬되지 않고 이어붙여짐에 주목!
pd.concat([df_a, df_b, df_c])

,key,num_a,num_b
0,a,1.0,NaN
1,b,2.0,NaN
2,c,3.0,NaN
3,d,4.0,NaN
4,e,5.0,NaN
0,a,NaN,11.0
1,b,NaN,15.0
2,d,NaN,35.0
3,f,NaN,45.0
4,g,NaN,55.0


In [10]:
# axis = 1인 경우 컬럼 역시 재정렬이나 병합 없이 이어붙여짐
pd.concat([df_a, df_b, df_c], axis = 1)

,key,num_a,key,num_b,key,num_a
0,a,1,a,11,f,6
1,b,2,b,15,g,7
2,c,3,d,35,h,8
3,d,4,f,45,i,9
4,e,5,g,55,j,0


### 4-2-2.merge() 함수
- dataframe1.merge(dataframe2) = dataframe1과 dataframe2를 병합
- how 인자를 통해 병합방식을 설정 가능
 - inner join: 머지 대상의 공통된 키만을 추려내 반환
 - left join: "왼쪽" df의 모든 키와 "오른쪽" df의 공통된 키를 반환
 - right join: "오른쪽" df의 모든 키와 "왼쪽" df의 공통된 키를 반환
 - outer join: 머지 대상의 모든 키를 na 포함하여 반환
- on 인자를 통해 병합의 기준이 되는 컬럼을 설정 가능:<br>미설정시 이름이 같은 컬럼들이 기준이 됨

In [11]:
# on 인자를 생략하여, 이름이 같은 "key" 컬럼이 기준이 됨

df_a.merge(df_b, how = 'outer')

,key,num_a,num_b
0,a,1.0,11.0
1,b,2.0,15.0
2,c,3.0,NaN
3,d,4.0,35.0
4,e,5.0,NaN
5,f,NaN,45.0
6,g,NaN,55.0


In [14]:
# 이름이 같은 컬럼이 복수일 경우, 일부만 키 컬럼으로 설정했을 때와 비교

df_a.merge(df_c, how = 'outer') # num_a 컬럼 역시 키 컬럼이며 병합의 대상이 아님

,key,num_a
0,a,1
1,b,2
2,c,3
3,d,4
4,e,5
5,f,6
6,g,7
7,h,8
8,i,9
9,j,0


In [16]:
df_a.merge(df_c, on = 'key', how = 'outer') # num_a 컬럼 역시 병합의 대상이 됨

,key,num_a_x,num_a_y
0,a,1.0,NaN
1,b,2.0,NaN
2,c,3.0,NaN
3,d,4.0,NaN
4,e,5.0,NaN
5,f,NaN,6.0
6,g,NaN,7.0
7,h,NaN,8.0
8,i,NaN,9.0
9,j,NaN,0.0


In [7]:
# "왼쪽"과 "오른쪽"의 df에 공통된 컬럼명이 없을 경우:
# 실습을 위해 df_b의 key 컬럼을 id로 변경한 df_d를 생성

df_d = df_b.rename(columns = {'key': 'id'})

In [8]:
# left_on과 right_on을 각각 지정

df_a.merge(df_d, left_on = "key", right_on = "id", how = "outer")

,key,num_a,id,num_b
0,a,1.0,a,11.0
1,b,2.0,b,15.0
2,c,3.0,NaN,NaN
3,d,4.0,d,35.0
4,e,5.0,NaN,NaN
5,NaN,NaN,f,45.0
6,NaN,NaN,g,55.0


### 4-2-3.join() 함수
- 구문은 merge와 비슷하나 결과는 concat과 비슷
 - 동일한 컬럼명이 있을 경우 에러 발생: 에러 처리 위해 lsuffix, rsuffix 지정 가능
 - 동일한 컬럼명의 내용을 기준으로 병합하고 싶다면 df.set_index('컬럼명')로 처리 필요

In [17]:
# "왼쪽"과 "오른쪽" df에 각각 접미사를 붙여 조인

df_a.join(df_b, lsuffix = "_a", rsuffix = "_b") # how 인자 설정 가능

,key_a,num_a,key_b,num_b
0,a,1,a,11
1,b,2,b,15
2,c,3,d,35
3,d,4,f,45
4,e,5,g,55


In [21]:
# 공통된 컬럼명을 가진 key를 인덱스로 처리하여 조인

df_a_join = df_a.set_index('key')
df_b_join = df_b.set_index('key')

df_a_join.join(df_b_join) # how 인자 미설정시 left join이 기본값

,num_a,num_b
key,,
a,1,11.0
b,2,15.0
c,3,NaN
d,4,35.0
e,5,NaN
